# Data USA API Pulls - Statewide

In order to pull median property value, median income, and the age breakdown of every small town in Montana, I will need to call each of them by their unique identifier from the DataUSA API. 

The unique identifiers for each "place" in Data USA's database is stored in the table on this page: https://datausa.io/about/classifications/Geography/Place

My first step will be to scrape the unique identifiers from that page. Then I will perform a V-Lookup in Excel with my list of Montana towns. Finally, I will perform the data pulls for each town.

In [17]:
# imports
import requests
import pandas as pd
from pandas.io.json import json_normalize
import csv
import json
import pandas as pd
import requests               
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [18]:
# The whole data table of identifiers is stored in a table with columns for identifiers and for places. 
# there are over 29k

url = 'https://datausa.io/about/classifications/Geography/Place'
r = requests.get(url)
r.status_code

200

In [7]:
soup = BeautifulSoup(r.text, 'html.parser')

From inspecting the website elements and the soup, I can see that all of the information is contained under the same tags. So, from my research on this issue, it looks like I need to parse the website with the script rather than spans and divs. 

In [14]:
//*[@id="Classifications"]/div[2]/div/div[1]/div[1]/div/div[2]/div[2]/div/div

[<div class="bp3-table-truncated-text">id</div>,
 <div class="bp3-table-truncated-text">16000US4200104</div>,
 <div class="bp3-table-truncated-text">id</div>]

In [23]:

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

table = driver.find_element(By.XPATH, '//*[@id="Classifications"]/div[2]/div/div[1]/div[1]/div/div[2]/div[2]/div/div')
ID = driver.find_element(By.XPATH, '//*[@id="Classifications"]/div[2]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[1]')
name = driver.find_element(By.XPATH, '//*[@id="Classifications"]/div[2]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[2]')

place_data = {
    'ID': ID.get_attribute('content'),
    'name': name.get_attribute('content'),
    'table': table.get_attribute('content')
}

print(place_data)
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/austinsmith/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


{'ID': None, 'name': None, 'table': None}


In [ ]:
# General process
# EXCEPT NEEDS TO BE EDITED SO THAT ALL OF THE DATAPOINTS ARE STORED IN THE SAME CSV!

# pull data
url_home = "https://datausa.io/api/data?measure=Property Value by Bucket,Property Value by Bucket%20Moe&geo=16000US3029950,01000US&drilldowns=Value%20Bucket"

# process
json_home = requests.get(url_home).json()

with open('g_home_data.json', 'w') as json_file :
    json.dump(json_home, json_file)
    
with open('g_home_data.json') as json_data :
    data = json.load(json_data)
    
# put it into a dataframe
home_df = pd.DataFrame(data['data'])

# save to csv
home_df.to_csv('g_home_data.csv')